<a href="https://colab.research.google.com/github/iceQHdrop/bioinformatic_data_mining/blob/main/ML_for_donar_finding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML for donar finding

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
! cp -r drive/MyDrive/Colab_Notebooks/bioinformatic_data_mining/dataset ./

In [62]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import re
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.optim as optim
import torch.nn as nn

## 数据读取

### 设定路径名

In [30]:
train_path = 'dataset/TrainingSet'
test_path = 'dataset/TestingSet'    
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)


### 读取训练数据

#### 定义读取函数

In [45]:
def LoadData(files, file_path):
    site_seqs = []
    normal_seqs = []

    print(f'Loading data from {file_path}...')

    for file in tqdm(files):
        with open(file_path + '/' + file, 'r') as f:
            text = f.readlines()
            site_positions = re.findall('(\d+)(?=,)', text[1])    # 提取位置
            seq = ''.join(text[2:]).replace('\n', '').lower()
        
            for position in site_positions:
                site_seqs.append(seq[int(position) - 4:int(position) + 5])
        
            for num in range(4):    # 提取非位点序列
                normal_position = np.random.randint(len(seq) - 9)    # 采样至倒数第九位
                normal_seq = seq[normal_position - 4:normal_position + 5]

                while normal_position in site_positions or \
                    set(normal_seq) != {'a', 't', 'c', 'g'} or \
                    len(normal_seq) != 9:    # 排除donar位点与'n', 'm' 等

                    normal_position = np.random.randint(len(seq) - 9)
                    normal_seq = seq[normal_position - 4:normal_position + 5]

                normal_seqs.append(normal_seq)
    
    site_df = pd.DataFrame(list(zip(site_seqs, np.ones(len(site_seqs)))),    # 位点
                            columns = ['Seq', 'Donar'])

    normal_df = pd.DataFrame(list(zip(normal_seqs, np.zeros(len(normal_seqs)))),    # 非位点
                              columns = ['Seq', 'Donar'])

    df = pd.concat([site_df, normal_df]).reset_index(drop = True)

    return df
  

#### 读取数据   

In [46]:
train_df = LoadData(train_files, train_path)
test_df = LoadData(test_files, test_path)

Loading data from dataset/TrainingSet...


100%|██████████| 462/462 [00:00<00:00, 5896.20it/s]


Loading data from dataset/TestingSet...


100%|██████████| 570/570 [00:00<00:00, 8562.44it/s]


## 特征提取

### One-hot 编码

In [47]:
X_train = train_df['Seq'].str.split('', expand = True).iloc[:, 1:10]
X_train = pd.get_dummies(X_train)
y_train = train_df['Donar']
X_train

,1_a,1_c,1_g,1_t,2_a,2_c,2_g,2_t,3_a,3_c,...,7_g,7_t,8_a,8_c,8_g,8_t,9_a,9_c,9_g,9_t
0,0,0,1,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
1,0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
2,0,0,0,1,0,1,0,0,0,1,...,1,0,1,0,0,0,0,0,1,0
3,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4224,1,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4225,0,0,1,0,1,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
4226,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
4227,0,0,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [48]:
X_test = test_df['Seq'].str.split('', expand = True).iloc[:, 1:10]
X_test = pd.get_dummies(X_test)
y_test = test_df['Donar']
X_test.head()

,1_a,1_c,1_g,1_t,2_a,2_c,2_g,2_t,3_a,3_c,...,7_g,7_t,8_a,8_c,8_g,8_t,9_a,9_c,9_g,9_t
0,0,1,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
1,0,0,1,0,0,0,1,0,1,0,...,1,0,1,0,0,0,0,0,1,0
2,0,1,0,0,1,0,0,0,1,0,...,1,0,1,0,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,0,0,0,1,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0


## SVM

In [49]:
clf = SVC(probability = True)

In [51]:
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])
auc_roc = auc(fpr, tpr)
auc_roc

0.9980254719289807

## ANN

### 确定设备

In [59]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The model will be running on {device}.')

The model will be running on cpu.


### 定义模型

In [60]:
class Net(nn.Module):
    def __init__(self, n_features, n_classes):
        super(Net, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_features, 100),
                                    nn.ReLU(True),
                                    nn.Linear(100, 500),
                                    nn.ReLU(True),
                                    nn.Linear(500, 500),
                                    nn.ReLU(True),
                                    nn.Linear(500, 100),
                                    nn.ReLU(True),
                                    nn.Linear(100, 2))
        
    def forward(self, X):
        return self.layers(X)

### 包装数据至DataSet

In [ ]:
whole_train_set = TensorDataset()

In [ ]:
def train(whole_train_set, model_name):
    train_set, valid_set = random_split(whole_train_set, [int(0.8*len(whole_train_set)),
                                                            len(whole_train_set) - int(0.8*len(whole_train_set))])
    train_loader = DataLoader(train_set, batch_size = int(0.2*len(train_set)), shuffle = True)
    valid_loader = DataLoader(valid_set, batch_size = int(0.5*len(valid_set)), shuffle = True)
    
    model = Net(500, 2)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    best_accuracy = 0
    
    for epoch in range(200):
        running_train_loss = 0.0  
        running_val_loss = 0.0
        correct, total = 0, 0 
        for i, data in enumerate(train_loader, 0):
            X = data[0].to(device)
            y = data[1].to(device)
            optimizer.zero_grad()
            
            outputs = model(X)  
            loss = criterion(outputs, y.long())
            loss.backward()
            optimizer.step()
            running_train_loss += loss
        
        train_loss = running_train_loss/len(train_loader)
        
        with torch.no_grad():
            model.eval()
            
            for i, data in enumerate(valid_loader, 0):
                X = data[0].to(device)
                y = data[1].to(device)
            
                outputs = model(X)
                loss = criterion(outputs, y.long())
                _, y_pred = torch.max(outputs, dim = 1)
                running_val_loss += loss
                total += outputs.size(0)
                correct += (y == y_pred).sum().item()
            
        val_loss = running_val_loss/len(valid_loader)
        
        accuracy = 100*correct/total
        
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), f'./best_model/{model_name}_best.pth')
            print('The model has been saved for the best accuracy %d %%'%(accuracy))
            best_accuracy = accuracy
        
        if epoch == 0:
            print('The model is working fine!')

        if (epoch + 1)%100 == 0:
            print('Completed training epoch', epoch + 1, 'Training Loss is: %.4f' %train_loss, 'Validation Loss is: %.4f' %val_loss, 'Accuracy is %d %%' % (accuracy))